In [1]:
import pandas as pd
import numpy as np
import os
import requests
from io import BytesIO
import json
import yadisk
from datetime import datetime, date, timedelta
import re
import gspread
from oauth2client.service_account import ServiceAccountCredentials

import config
from yandex_disk_func import *
from parse_functions import *
from google_connector import *
from date_time_func import *

# # указываем путь и название файла с токеном для Яндекс Диск
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\35_mediaplan_parser'
# забираем токен для подключения к гугл
service_key = config.service
google_sheet_link = 'https://docs.google.com/spreadsheets/d/1eakG0BJF1eSyi0hyGUpdw166TH_C9h_2zjnyrsMeejk/edit?usp=sharing' #'https://docs.google.com/spreadsheets/d/11hF_txYEbKrQImRQ0x3Fia3PyJvDAqtpGI25idV7oa4/edit?usp=sharing'
gmail = config.gmail
sheet_name = 'prog'

# обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета
public_key = config.public_key 

# забираем список папок в нужной нам директории
res = get_yandex_disk_folders(public_key)
yandex_folders = res.json() # парсим ответ

200


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None
# pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
# создаем словарь, где сохраним каждый продукт в отдельный датаФрейм со своим ключом
main_dict = {}

In [4]:
get_data_from_ya_folder(yandex_folders, main_dict, flag='prog')                     


/01_yandex
/02_prog
beeline_plan_игроник_ингосстрах_кис_лето_banners_020724
firstdata_игроник_ингосстрах_срм_media_28.08.2024
    mediaplan без ск
hybrid_бронь_игроник_национальная лотерея_день рождения нл_klientskiy_plan_20240920 (1)
    медиаплан
mp_mobidriven_игроник_ингосстрах_инго экосистема_05.09
    мп
mp_roxot_игроник_psb_brand-booster_sep-dec24
    псб bb banner
    псб bb olv
ингосстрах игроник_weborama_banners_24092024 (1)
    banners
ингосстрах_инго экосистемы_игроник_segmento _2024-09-05
/03_Полина


In [5]:
df = pd.concat(main_dict, ignore_index=True)
df = df.fillna('')

df['views'] = df['views'].astype('float').round(0)
int_columns = ['reach', 'impressions', 'clicks', 'views']
df[int_columns] = df[int_columns].apply(lambda x: x.astype('int'))

In [6]:
float_columns = ['unit price', 'budget_without_nds', 'budget_nds', 'vtr, %']
df[float_columns] = df[float_columns].apply(lambda x: x.astype('float')).round(2)

In [7]:
df.columns

Index(['supplier', 'report_name', 'sheet_name', 'brand', 'period', 'source',
       'site/ssp', 'placement', 'targeting', 'geo', 'soc_dem',
       'ad copy format', 'rotation type', 'unit price', 'frequency', 'reach',
       'impressions', 'clicks', 'budget_without_nds', 'budget_nds', 'views',
       'vtr, %'],
      dtype='object')

In [8]:
# download_columns_list = ['supplier', 'report_name', 'sheet_name', 'brand', 'geo', 'soc_dem', 'ad copy format', 'rotation type', 'frequency', 'reach',
# 'impressions', 'clicks', 'budget_without_nds', 'budget_nds', 'views', 'vtr, %']

In [9]:
download_columns_list = df.columns

In [10]:
final_df = df[download_columns_list]

In [11]:
final_df

,supplier,report_name,sheet_name,brand,period,source,site/ssp,placement,targeting,geo,soc_dem,ad copy format,rotation type,unit price,frequency,reach,impressions,clicks,budget_without_nds,budget_nds,views,"vtr, %"
0,beeline,beeline_plan_игроник_ингосстрах_кис_лето_banners_020724,Plan_Media,Ингосстрах_КИС_Лето,15.07-14.08.24,beeline AI,DSP,Desktop + mobile \n(+in-app),"ГЕО: РФ \nЦА: Все 18+\nПользователи, кто заходил на сайты/установлено приложение конкурентов в категории страхования (Росгосстрах, АльфаСтрахование, Тинькофф Страхование, СберСтрахование) и в банковской категории (Сбер, Альфа, Тинькофф, Россгосстрах, Согаз и т.д.)",рф,все 18+,Banners,CPM,88.00,3,7666666,23000000,29900,2024000.00,2428800.00,0,0.00
1,beeline,beeline_plan_игроник_ингосстрах_кис_лето_banners_020724,Plan_Media,Ингосстрах_КИС_Лето,15.07-14.08.24,beeline AI,DSP,Desktop + mobile \n(+in-app),ГЕО: РФ\nЦА: Все 18+\nКлиенты Сбера по транзакционным смс и установленному приложению,рф,все 18+,Banners,CPM,88.00,3,10000000,30000000,39000,2640000.00,3168000.00,0,0.00
2,firstdata,firstdata_игроник_ингосстрах_срм_media_28.08.2024,mediaplan без ск,Ингосстрах,3 months,Платформа First Data,,,"ГЕО: РФ\nЦА: МЖ. Доход ВС. \nПокупатели полисов КАСКО за 10-12 месяцев до старта рк. Владельцы авто на основании данных о покупках: автозапчастей, посещений автоателье, автосервисов, оплаты штрафов за эвакуацию.\nПокупатели услуг дизайнера интерьера, строительных компаний; товаров для строительства и ремонта: окна, двери, плитка, сантехника, напольные покрытия, краски, обои тд. Покупатели товаров для загородного дома: камины, барбекю, мангалы, газон, беседки, мебель уличная, газонокосилки и тд. \nИсточники: ОФД\n",рф,мж. доход вс.,Видео до 20сек.,CPM,175.00,5,3110000,15550000,31100,2721250.00,3265500.00,4665000,0.30
3,firstdata,firstdata_игроник_ингосстрах_срм_media_28.08.2024,mediaplan без ск,Ингосстрах,3 months,Платформа First Data,,,"ГЕО: РФ\nЦА: МЖ. Доход ВС. \nПокупатели полисов КАСКО за 10-12 месяцев до старта рк. Владельцы авто на основании данных о покупках: автозапчастей, посещений автоателье, автосервисов, оплаты штрафов за эвакуацию.\nПокупатели услуг дизайнера интерьера, строительных компаний; товаров для строительства и ремонта: окна, двери, плитка, сантехника, напольные покрытия, краски, обои тд. Покупатели товаров для загородного дома: камины, барбекю, мангалы, газон, беседки, мебель уличная, газонокосилки и тд. \nИсточники: ОФД\n",рф,мж. доход вс.,Баннеры / Универсальные Баннеры,CPM,130.00,5,4000000,20000000,40000,2600000.00,3120000.00,0,0.00
4,hybrid,hybrid_бронь_игроник_национальная лотерея_день рождения нл_klientskiy_plan_20240920 (1),медиаплан,Национальная лотерея,26.08.2024 - 28.09.2024,hybrid,,Desktop+Mobile (web + in-app)***,"Национальная лотерея\nРК ""День Рождения НЛ""\n\nЦА: МЖ 20+\n\n1. Аудиторные сегменты+Кастомные сегменты Hybrid Segments\n2. Hybrid Semantica \n3. Hybrid Places+SuperGeo на посетителей точек продаж столото, на посетителей пунктов выдачи маркетплейсов\n4. Сбор аудитории с сайтов во вкладке ""Пример сайтов""",РФ,мж 20+,Video \n(хрон. 15 и 6 сек),CPM,330.00,3,1490236,4470708,35765,1475333.64,1770400.37,3129496,0.70
5,hybrid,hybrid_бронь_игроник_национальная лотерея_день рождения нл_klientskiy_plan_20240920 (1),медиаплан,Национальная лотерея,26.08.2024 - 28.09.2024,hybrid,,Desktop+Mobile (web + in-app)***,"Национальная лотерея\nРК ""День Рождения НЛ""\n\nЦА: МЖ 20+\n\n1. Аудиторные сегменты+Кастомные сегменты Hybrid Segments\n2. Hybrid Semantica \n3. Hybrid Places+SuperGeo на посетителей точек продаж столото, на посетителей пунктов выдачи маркетплейсов\n4. Сбор аудитории с сайтов во вкладке ""Пример сайтов""",РФ,мж 20+,"Banner, Native",CPC,44.00,3,1250000,3750000,7500,330000.00,396000.00,0,0.00
6,mobidriven,mp_mobidriven_игроник_ингосстрах_инго экосистема_05.09,мп,Ингосстрах,01.10-31.10,Mobidriven,,in web,All 18+ (30% мужчины; 70% женщины); следят за собой; покупают косметику и средства для ухода за телом + посетители сайтов (вкладка) (данн

In [12]:
# создаем подключение к Гуглу
client = create_connection(service_key)

Connection established successfully...


In [13]:
# прописываем путь и подключаемсяк нужному листу Гугл докса
sh = client.open_by_url(google_sheet_link)
sh.share(gmail, perm_type='user', role='writer')
google_sheet = sh.worksheet(sheet_name)

In [14]:
# выгружем наш датаФрейм на лист
google_sheet.clear()
export_dataframe_to_google_sheet(google_sheet, final_df)

DataFrame exported successfully...


In [15]:
# создаем кастомный календарь для медиаплана
sheet_name = 'weekYear'
calendar_df = get_mediaplan_calendar(cur_year='2024', start_day_num=0)

start: 2024-01-01, end: 2024-12-31


In [16]:
# создаем подключение к Гуглу
client = create_connection(service_key)

Connection established successfully...


In [17]:
# прописываем путь и подключаемсяк нужному листу Гугл докса
sh = client.open_by_url(google_sheet_link)
sh.share(gmail, perm_type='user', role='writer')
google_sheet = sh.worksheet(sheet_name)

In [18]:
# выгружем наш датаФрейм на лист
google_sheet.clear()
export_dataframe_to_google_sheet(google_sheet, calendar_df)

DataFrame exported successfully...
